In [1]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
import numpy as np
import math
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from PIL import Image
import random
import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.models import Sequential
import keras.backend as K

Using TensorFlow backend.


In [2]:
def file2array(path, delimiter):
    recordlist = []
    fp = open(path, 'r', encoding='utf-8')
    content = fp.read()     # content现在是一行字符串，该字符串包含文件所有内容
    fp.close()
    rowlist = content.splitlines()  # 按行转换为一维表，splitlines默认参数是‘\n’
    # 逐行遍历
    # 结果按分隔符分割为行向量
    recordlist = [row.split(delimiter) for row in rowlist if row.strip()]
    return np.array(recordlist)

In [3]:
#import numpy as np
import scipy.ndimage


def ndarray_zoom_scaling(label, new_h, new_w):
    """
    Implement scaling for ndarray with scipy.ndimage.zoom
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_zoom_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 2 2 2 2 3 3 3]
     [1 1 1 2 2 2 2 3 3 3]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [7 7 7 8 8 8 8 9 9 9]
     [7 7 7 8 8 8 8 9 9 9]]
    """
    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]
    if len(label.shape) == 2:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w], order=0)
    else:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w, 1], order=0)
    return label_new


def ndarray_nearest_neighbour_scaling(label, new_h, new_w):
    """
    Implement nearest neighbour scaling for ndarray
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_nearest_neighbour_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [7 7 7 7 8 8 8 9 9 9]
     [7 7 7 7 8 8 8 9 9 9]]
    """
#    if len(label.shape) == 2:
#        label_new = np.zeros([new_h, new_w], dtype=label.dtype)
#    else:
#        label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)
    label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)

    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]

    y_pos = np.arange(new_h)
    x_pos = np.arange(new_w)
    y_pos = np.floor(y_pos / scale_h).astype(np.int32)
    x_pos = np.floor(x_pos / scale_w).astype(np.int32)

    y_pos = y_pos.reshape(y_pos.shape[0], 1)
    y_pos = np.tile(y_pos, (1, new_w))
    x_pos = np.tile(x_pos, (new_h, 1))
    assert y_pos.shape == x_pos.shape

    label_new[:, :] = label[y_pos[:, :], x_pos[:, :]]
    return label_new


In [4]:
def DataSet(iFold):
    
    train_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_samples_train_F%d") % (iFold, iFold)    
    test_path  =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_samples_test_F%d") % (iFold, iFold)
    #
    data_train = file2array(train_path, " ")
    nrow_train = data_train.shape[0]
    ncol_train = data_train.shape[1]
    #
    data_test  = file2array(test_path, " ")
    nrow_test  = data_test.shape[0]
    ncol_test  = data_test.shape[1]
    #
    if ncol_train != ncol_test:
        print('Numbers of features are not consistant.')
    if nrow_train + nrow_test != 1567:
        print('Numbers of samples are not consistant.')
    #
    train_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_labels_train_F%d") % (iFold, iFold)    
    test_path  =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_labels_test_F%d") % (iFold, iFold)
    #
    lab_train = file2array(train_path, " ")
    #
    lab_test  = file2array(test_path, " ")
    #
    num_feature = ncol_train
    side_len  = math.ceil(math.sqrt(num_feature))
    num_class = 2
        
    X_train = np.empty((nrow_train, side_len, side_len, 1))
    Y_train = np.empty((nrow_train, num_class))

    for index in range(nrow_train):
        tmp_cnt = 0
        for i in range(side_len):
            for j in range(side_len):
                if tmp_cnt < num_feature:
                    X_train[index,i,j,0] = data_train[index,tmp_cnt]
                else:
                    X_train[index,i,j,0] = 0.5
                tmp_cnt = tmp_cnt + 1
    
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_train[index] = tmp_v
    
    X_test  = np.empty((nrow_test, side_len, side_len, 1))
    Y_test  = np.empty((nrow_test, num_class))

    for index in range(nrow_test):
        tmp_cnt = 0
        for i in range(side_len):
            for j in range(side_len):
                if tmp_cnt < num_feature:
                    X_test[index,i,j,0] = data_test[index,tmp_cnt]
                else:
                    X_test[index,i,j,0] = 0.5
                tmp_cnt = tmp_cnt + 1
    
    for index in range(nrow_test):
        tmp = lab_test[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_train[index] = tmp_v
    
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]
    
    return X_train,Y_train,X_test,Y_test

In [5]:
#import tensorflow as tf
def LeakyRelu(x, leak=0.01, name="LeakyRelu"):
   with tf.variable_scope(name):
     f1 = 0.5 * (1 + leak)
     f2 = 0.5 * (1 - leak)
     return f1 * x + f2 * tf.abs(x)

In [8]:
def neural(channel,height,width,classes):
    input_shape = (channel,height,width)
    if K.image_data_format() == "channels_last":#确认输入维度,就是channel是在开头，还是结尾
        input_shape = (height,width,channel)
    model = Sequential()#顺序模型（keras中包括顺序模型和函数式API两种方式）
    model.add(Conv2D(6,(3,3),padding="valid",activation=LeakyRelu,input_shape=input_shape,name="conv1"))
    #model.add(Conv2D(6,(3,3),padding="valid",input_shape=input_shape,name="conv1"))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(AveragePooling2D(pool_size=(2,2),strides=(2,2),name="pool1"))
    model.add(Conv2D(12,(3,3),padding="valid",activation=LeakyRelu,name="conv2",))
    #model.add(Conv2D(12,(3,3),padding="valid",name="conv2",))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(AveragePooling2D(pool_size=(2,2),strides=(2,2),name="pool2"))
    model.add(Flatten())
    model.add(Dense(192,activation="relu",name="fc1"))
    model.add(Dense(classes,activation="softmax",name="fc2"))
    return model


In [9]:
models = []
for fold in range(10):
    iFold = fold + 1
    X_train,Y_train,X_test,Y_test = DataSet(iFold)
    print('X_train shape : ',X_train.shape)
    print('Y_train shape : ',Y_train.shape)
    print('X_test shape : ',X_test.shape)
    print('Y_test shape : ',Y_test.shape)
    #print(X_train[0])
    #print(Y_train[0,1])

    # # model
    #model = ResNet50(
    #    weights=None,
    #    classes=2
    #)
    
    ##
    model = neural(1,22,22,2)

    model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # # train
    model.fit(X_train, Y_train, epochs=100, batch_size=6)

    # # evaluate
    model.evaluate(X_test, Y_test, batch_size=32)
    
    # #
    models.append(model)

    # # save
    save_path =("./my_resnet_model.h5_F%d") % (iFold)
    model.save(save_path)

X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
Epoch 1/100
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2495 - acc: 0.9348
Epoch 2/100
1410/1410 [==============================] - 1s 572us/step - loss: 0.2273 - acc: 0.9418
Epoch 3/100
1410/1410 [==============================] - 1s 601us/step - loss: 0.2261 - acc: 0.9418
Epoch 4/100
1410/1410 [==============================] - 1s 565us/step - loss: 0.2291 - acc: 0.9418
Epoch 5/100
1410/1410 [==============================] - 1s 579us/step - loss: 0.2258 - acc: 0.9418
Epoch 6/100
1410/1410 [==============================] - 1s 579us/step - loss: 0.2272 - acc: 0.9418
Epoch 7/100
1410/1410 [==============================] - 1s 587us/step - loss: 0.2264 - acc: 0.9418
Epoch 8/100
1410/1410 [==============================] - 1s 574us/step - loss: 0.2246 - acc: 0.9418
Epoch 9/100
1410/1410 [==============================] - 1s 607us/step - loss: 0.2

D:\toolkits.win\Anaconda3-4.4.0\envs\dlwin36-CPU\lib\site-packages\keras\models.py:124: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
Epoch 1/100
1410/1410 [==============================] - 1s 829us/step - loss: 0.2504 - acc: 0.9333
Epoch 2/100
1410/1410 [==============================] - 1s 749us/step - loss: 0.2310 - acc: 0.9418
Epoch 3/100
1410/1410 [==============================] - 1s 738us/step - loss: 0.2315 - acc: 0.9418
Epoch 4/100
1410/1410 [==============================] - 1s 757us/step - loss: 0.2272 - acc: 0.9418
Epoch 5/100
1410/1410 [==============================] - 1s 737us/step - loss: 0.2263 - acc: 0.9418
Epoch 6/100
1410/1410 [==============================] - 1s 735us/step - loss: 0.2263 - acc: 0.9418
Epoch 7/100
1410/1410 [==============================] - 1s 739us/step - loss: 0.2266 - acc: 0.9418
Epoch 8/100
1410/1410 [==============================] - 1s 737us/step - loss: 0.2244 - acc: 0.9418
Epoch 9/100
1410/1410 [==============================] - 1s 827us/step - loss: 0

1410/1410 [==============================] - 1s 849us/step - loss: 0.0441 - acc: 0.9837
Epoch 82/100
1410/1410 [==============================] - 1s 840us/step - loss: 0.0322 - acc: 0.9894
Epoch 83/100
1410/1410 [==============================] - 1s 835us/step - loss: 0.0210 - acc: 0.9943
Epoch 84/100
1410/1410 [==============================] - 1s 850us/step - loss: 0.0214 - acc: 0.9936
Epoch 85/100
1410/1410 [==============================] - 1s 838us/step - loss: 0.0253 - acc: 0.9929
Epoch 86/100
1410/1410 [==============================] - 1s 823us/step - loss: 0.0173 - acc: 0.9965
Epoch 87/100
1410/1410 [==============================] - 1s 815us/step - loss: 0.0082 - acc: 0.9993
Epoch 88/100
1410/1410 [==============================] - 1s 826us/step - loss: 0.0083 - acc: 0.9993
Epoch 89/100
1410/1410 [==============================] - 1s 819us/step - loss: 0.0330 - acc: 0.9901
Epoch 90/100
1410/1410 [==============================] - 1s 807us/step - loss: 0.0299 - acc: 0.9894
Epo

Epoch 40/100
1410/1410 [==============================] - 1s 886us/step - loss: 0.1243 - acc: 0.9518
Epoch 41/100
1410/1410 [==============================] - 1s 907us/step - loss: 0.1204 - acc: 0.9539
Epoch 42/100
1410/1410 [==============================] - 1s 885us/step - loss: 0.1224 - acc: 0.9532
Epoch 43/100
1410/1410 [==============================] - 1s 892us/step - loss: 0.1156 - acc: 0.9560
Epoch 44/100
1410/1410 [==============================] - 1s 905us/step - loss: 0.1111 - acc: 0.9589
Epoch 45/100
1410/1410 [==============================] - 1s 901us/step - loss: 0.1068 - acc: 0.9582
Epoch 46/100
1410/1410 [==============================] - 1s 883us/step - loss: 0.1062 - acc: 0.9596
Epoch 47/100
1410/1410 [==============================] - 1s 856us/step - loss: 0.1029 - acc: 0.9610
Epoch 48/100
1410/1410 [==============================] - 1s 888us/step - loss: 0.0987 - acc: 0.9603
Epoch 49/100
1410/1410 [==============================] - 2s 1ms/step - loss: 0.0966 - acc:

157/157 [==============================] - 0s 623us/step
X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
Epoch 1/100
1410/1410 [==============================] - 1s 1ms/step - loss: 0.2414 - acc: 0.9390
Epoch 2/100
1410/1410 [==============================] - 1s 866us/step - loss: 0.2322 - acc: 0.9411
Epoch 3/100
1410/1410 [==============================] - 1s 890us/step - loss: 0.2310 - acc: 0.9411
Epoch 4/100
1410/1410 [==============================] - 1s 865us/step - loss: 0.2319 - acc: 0.9411
Epoch 5/100
1410/1410 [==============================] - 1s 878us/step - loss: 0.2301 - acc: 0.9411
Epoch 6/100
1410/1410 [==============================] - 1s 862us/step - loss: 0.2310 - acc: 0.9411
Epoch 7/100
1410/1410 [==============================] - 1s 869us/step - loss: 0.2287 - acc: 0.9411
Epoch 8/100
1410/1410 [==============================] - 1s 846us/step - loss: 0.2276 - acc: 0.9411
Epoch 9/100
1410/1410 [==

1410/1410 [==============================] - 1s 914us/step - loss: 0.0831 - acc: 0.9660
Epoch 59/100
1410/1410 [==============================] - 1s 847us/step - loss: 0.0761 - acc: 0.9738
Epoch 60/100
1410/1410 [==============================] - 1s 877us/step - loss: 0.0694 - acc: 0.9652
Epoch 61/100
1410/1410 [==============================] - 1s 861us/step - loss: 0.0653 - acc: 0.9695
Epoch 62/100
1410/1410 [==============================] - 1s 900us/step - loss: 0.0494 - acc: 0.9780
Epoch 63/100
1410/1410 [==============================] - 1s 855us/step - loss: 0.0499 - acc: 0.9823
Epoch 64/100
1410/1410 [==============================] - 1s 871us/step - loss: 0.0473 - acc: 0.9858
Epoch 65/100
1410/1410 [==============================] - 1s 854us/step - loss: 0.0392 - acc: 0.9865
Epoch 66/100
1410/1410 [==============================] - 1s 891us/step - loss: 0.0442 - acc: 0.9858
Epoch 67/100
1410/1410 [==============================] - 1s 877us/step - loss: 0.0444 - acc: 0.9851
Epo

1411/1411 [==============================] - 1s 804us/step - loss: 0.2026 - acc: 0.9419
Epoch 17/100
1411/1411 [==============================] - 1s 810us/step - loss: 0.1983 - acc: 0.9419
Epoch 18/100
1411/1411 [==============================] - 1s 803us/step - loss: 0.1924 - acc: 0.9419
Epoch 19/100
1411/1411 [==============================] - 1s 823us/step - loss: 0.1944 - acc: 0.9419
Epoch 20/100
1411/1411 [==============================] - 1s 830us/step - loss: 0.1867 - acc: 0.9419
Epoch 21/100
1411/1411 [==============================] - 1s 829us/step - loss: 0.1896 - acc: 0.9419
Epoch 22/100
1411/1411 [==============================] - 1s 813us/step - loss: 0.1815 - acc: 0.9419
Epoch 23/100
1411/1411 [==============================] - 1s 810us/step - loss: 0.1739 - acc: 0.9419
Epoch 24/100
1411/1411 [==============================] - 1s 798us/step - loss: 0.1743 - acc: 0.9426
Epoch 25/100
1411/1411 [==============================] - 1s 788us/step - loss: 0.1739 - acc: 0.9440
Epo

1411/1411 [==============================] - 1s 776us/step - loss: 0.0020 - acc: 1.0000
Epoch 98/100
1411/1411 [==============================] - 1s 803us/step - loss: 0.0015 - acc: 1.0000
Epoch 99/100
1411/1411 [==============================] - 1s 768us/step - loss: 0.0015 - acc: 1.0000
Epoch 100/100
156/156 [==============================] - 0s 830us/step
X_train shape :  (1411, 22, 22, 1)
Y_train shape :  (1411, 2)
X_test shape :  (156, 22, 22, 1)
Y_test shape :  (156, 2)
Epoch 1/100
1411/1411 [==============================] - 1s 987us/step - loss: 0.2393 - acc: 0.9376
Epoch 2/100
1411/1411 [==============================] - 1s 751us/step - loss: 0.2313 - acc: 0.9419
Epoch 3/100
1411/1411 [==============================] - 1s 762us/step - loss: 0.2309 - acc: 0.9419
Epoch 4/100
1411/1411 [==============================] - 1s 763us/step - loss: 0.2266 - acc: 0.9419
Epoch 5/100
1411/1411 [==============================] - 1s 796us/step - loss: 0.2258 - acc: 0.9419
Epoch 6/100
1411/14

In [10]:
X_train,Y_train,X_test,Y_test = DataSet(1)
true_labels = np.zeros(len(Y_test))
print(len(Y_test))

157


In [13]:
all_precisions=[]
all_recalls=[]
all_Fvalues=[]
nRun = 10
num_class = 2
for iRun in range(nRun):
    iFold = iRun + 1
    X_train,Y_train,X_test,Y_test = DataSet(iFold)
    print('X_train shape : ',X_train.shape)
    print('Y_train shape : ',Y_train.shape)
    print('X_test shape : ',X_test.shape)
    print('Y_test shape : ',Y_test.shape)
    #print(X_train[0])
    #print(Y_train[0,1])
    
    test_rows = len(Y_test)
    
    tmp_precision = np.zeros(num_class)
    tmp_recall    = np.zeros(num_class)
    tmp_Fvalue    = np.zeros(num_class)
    tmp_beta = 1
    model = models[iRun]
    yhat_label = model.predict_classes(X_test, batch_size=32, verbose=1)
    true_labels = np.zeros(len(Y_test))
    for index in range(len(Y_test)):
        tmp_vec = Y_test[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        true_labels[index] = tmp_ind
    
    N_TP = np.zeros(num_class)
    N_FP = np.zeros(num_class)
    N_TN = np.zeros(num_class)
    N_FN = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = true_labels[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    N_TP[j]=N_TP[j]+1
                else:
                    N_FP[j]=N_FP[j]+1
            else:
                if j == true_label:
                    N_FN[j]=N_FN[j]+1
                else:
                    N_TN[j]=N_TN[j]+1
            
    for i in range(num_class):
        if N_TP[i] > 0:
            tmp_precision[i] = N_TP[i] / (N_TP[i] + N_FP[i])
            tmp_recall[i]    = N_TP[i] / (N_TP[i] + N_FN[i])
            tmp_Fvalue[i]=(1 + tmp_beta*tmp_beta)*tmp_recall[i]*tmp_precision[i]/\
                          (tmp_beta*tmp_beta*(tmp_recall[i]+tmp_precision[i]))
        else:
            tmp_precision[i] = 0
            tmp_recall[i] = 0
            tmp_Fvalue[i] = 0
    
    all_precisions.append(tmp_precision)
    all_recalls.append(tmp_recall)
    all_Fvalues.append(tmp_Fvalue)


X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 108us/step
X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 718us/step
X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 719us/step
X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 737us/step
X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 718us/step
X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157

In [20]:
for iRun in range(nRun):
    print(all_precisions[iRun])
    print(all_recalls[iRun])
    print(all_Fvalues[iRun])


[0.86363636 0.66666667]
[0.99253731 0.08695652]
[0.92361111 0.15384615]
[1. 0.]
[0.99363057 0.        ]
[0.99680511 0.        ]
[0.09271523 0.66666667]
[0.875      0.02836879]
[0.16766467 0.05442177]
[0.12987013 1.        ]
[1.         0.02189781]
[0.22988506 0.04285714]
[0.84931507 0.09090909]
[0.92537313 0.04347826]
[0.88571429 0.05882353]
[0.19444444 0.92307692]
[0.96551724 0.09375   ]
[0.32369942 0.17021277]
[0.86092715 0.33333333]
[0.97014925 0.08695652]
[0.9122807  0.13793103]
[0.12582781 1.        ]
[1.         0.03649635]
[0.22352941 0.07042254]
[0.20645161 1.        ]
[1.         0.00806452]
[0.34224599 0.016     ]
[0.16788321 0.78947368]
[0.85185185 0.11627907]
[0.2804878 0.2027027]


In [21]:
tmp=np.zeros([3,num_class])
print(tmp)
print(tmp[0,0])

all_indices=[]
all_indices.append(all_precisions)
all_indices.append(all_recalls)
all_indices.append(all_Fvalues)

print(all_precisions[0][0])
print(all_indices[0][0][0])

[[0. 0.]
 [0. 0.]
 [0. 0.]]
0.0
0.8636363636363636
0.8636363636363636


In [22]:
mean_indices=np.zeros([3,num_class])
min_indices=np.ones([3,num_class])
max_indices=np.zeros([3,num_class])
std_indices=np.zeros([3,num_class])

for iInd in range(3):
    for iRun in range(nRun):
        for i in range(num_class):
            mean_indices[iInd,i] = mean_indices[iInd,i] + all_indices[iInd][iRun][i]
            if min_indices[iInd,i] > all_indices[iInd][iRun][i]:
                min_indices[iInd,i] = all_indices[iInd][iRun][i]
            if max_indices[iInd,i] < all_indices[iInd][iRun][i]:
                max_indices[iInd,i] = all_indices[iInd][iRun][i]

for iInd in range(3):
    for i in range(num_class):
        mean_indices[iInd,i] = mean_indices[iInd,i] / nRun
                
for iInd in range(3):
    for iRun in range(nRun):
        for i in range(num_class):
            std_indices[iInd,i] = (mean_indices[iInd,i] - all_indices[iInd][iRun][i])*(mean_indices[iInd,i] - all_indices[iInd][iRun][i])

for iInd in range(3):
    for i in range(num_class):
        std_indices[iInd,i] = std_indices[iInd,i] / nRun
        std_indices[iInd,i] = std_indices[iInd,i] ** 0.5


In [23]:
print(mean_indices)
print(min_indices)
print(max_indices)
print(std_indices)


[[0.4491071  0.64701264]
 [0.95740594 0.05222478]
 [0.52859236 0.09072176]]
[[0.09271523 0.        ]
 [0.85185185 0.        ]
 [0.16766467 0.        ]]
[[1.         1.        ]
 [1.         0.11627907]
 [0.99680511 0.2027027 ]]
[[0.0889308  0.04505014]
 [0.03337913 0.02025574]
 [0.07845755 0.03541148]]


In [ ]:
# # restore
#model = tf.keras.models.load_model('my_resnet_model.h5')

# # test
#img_path = "../my_nn/dataset/test/medicine/IMG_20190717_135408_BURST91.jpg"
#img_path = "../my_nn/dataset/test/glue/IMG_20190717_135425_BURST91.jpg"

#img = image.load_img(img_path, target_size=(224, 224))

#plt.imshow(img)
#img = image.img_to_array(img)/ 255.0
#img = np.expand_dims(img, axis=0)  # 为batch添加第四维

#print(model.predict(img))
#np.argmax(model.predict(img))